# Battle of the Neighbourhoods Week 2 - Report

## 1. Introduction/Business Problem

### 1.1 Introduction  

As the owner of a hair/nail salon, I am looking to figure out what neighbourhood would work best as the location of my new business.

Hair/nail salons have a specific clientele, wanting a higher number of female residents as well as residents with disposable income to spare. So ideally the neighbourhood for this new business would have to have a high enough female population and a medium to high average income. However, it would be good to see the actual data around where salons exist, especially if it counters that concept. Perhaps we'll see that lower income areas are more welcoming for a salon.

Additionally, I wouldn't want to place my business in a location that is already filled with salons. A new business might not do well with stiff competition who might already have loyal clients that can't be swayed away. However, it also might not be good to start a salon in an area with absolutely no salons at all, as there might be a reason why that type of business has failed in the past.

Information on how salons are already spread throughout the city, along with information like income, could help me figure out what part of the city would work best for my new business.

## 2. Data

### 2.1 Data Sources

Data on neighbourhood businesses can be found on FourSquare, with the categories telling what type of business it is. There are two types of salon listed, and seeing where those categories line up compared to everything else can be useful in determining a neighbourhood

Data on population demographics, such as population, average income, female population and more can be found on Toronto's Open Data Portal: https://open.toronto.ca/

Most data sets are available to download as a csv or excel file, and there is fairly recent neighbourhood and demographics data that I pulled for use.

The first dataset is a simple geographic dataset with coordinates for all 140 Toronto neighbourhoods. This will be useful for sending to Foursquare to get venue information, as latitude and longitude are two variable that we pass into the GET function. It can be found here:https://open.toronto.ca/dataset/neighbourhoods/

The second dataset is a set of census data from 2008 and 2011, with details on each Toronto neighbourhood with information like: total population, male/female population, average income, total area, education, employed/unemployed, etc. This can be useful in comparing against what types of venues are popular in each neighbourhood and cluster. It can be found here: https://open.toronto.ca/dataset/wellbeing-toronto-demographics/

### 2.2 Data Cleaning

The first thing I did was pull the geographic neighbourhood data from the url, pull the useful columns to a new dataframe and sort it alphabetically (to more easily match with data later on) using pandas.read_csv. 

Included features: neighbourhood name, code, longititude, and latitude. 

Sample discarded features included: AREA_ID, OBJECTID, Shape_Area, Shape_Length, geometry. These were discarded due to being meant for drawing the borders of individual neighbourhoods, which was beyond the scope of this problem.

<div>
<img src="./img/neigh-data-pull.png" width="400">
</div>

I then pulled the demographic data using pandas.read_csv, and put the relevant columns in a new dataframe. 

Included features: total population, male population, female population, and average family income.

Sample discarded features: Home Repairs Needed, Low Income Children, Household Income Category, Tenant Average Rent. These were discarded due to either incomplete data or lack of relevance to the problem. I used Average Income as my economic indicator, as it had complete data for every neighbourhood.

I merged my two datasets into one dataframe, using the Neighbourhood Code as my matching column.

<div>
<img src="./img/neigh_demo_data.png" width="600">
</div>

I then pulled venue data from Foursquare. This was done using the FourSquare API, and sending a list of neighbourhood coordinates and requesting a list of the top 100 venues within 1km of that location. I then used a **For** loop to place the json data into a dataframe. The resulting data looked as follows:

<div>
<img src="./img/venue_pull.png" width="800">
</div>

We can see here that neighbourhoods can have multiple venues, each with a different category. Despite only have 140 neighbourhoods, there are 2020 venues in the dataframe. This will need to be manipulated in some way to be useful for our clustering algorithm later on.

So, I used one hot encoding to turn the list of Venue Categories into columns with a binary value indicating wether the venue was of that category (0 = it is not of the category, 1 = it is of that category). I then grouped the rows by neighbourhood, so that each column would show the total number of venues in the neighbourhood of that venue category. I merged the demographics information as well.

I also filled in any missing values with zeroes. There were missing values due to some neighbourhoods not returning any venues from the FourSquare request.

<div>
<img src="./img/one-hot-sum.png" width="1000">
</div>

I then normalized all my numerical data so that the large values of my demographics data didn't overwhelm the model with respect to the much smaller venue values. I did this using scikit-learn's proprocessing module.

<div>
<img src="./img/normalized-data.png" width="600">
</div>

I then did the same data cleaning on a subset of my venue data. Instead of considering every venue category, I narrowed it down to just the categories "Nail Salon" and "Salon / Barbershop", as these would be my business' direct competition. I wanted to later on do a clustering algorithm on this subset, to see if there are any differences. The resulting data is as follows:

<div>
<img src="./img/venues_data_salon.png" width="1000">
</div>

## 3. Methodology

### 3.1 Exploratory Data Analysis

I did some exploratory data analysis on my neighbourhood demographics data.

I created a histogram of the populations of the Toronto neighbourhoods. We can see that most of the neighbourhoods have a population between 9492 and 25659, with a few outliers with large populations.

<div>
<img src="./img/population-hist.png" width="600">
</div>

I also created a histogram of the female populations of the neighbourhoods, as most of my business' clients will likely be female. We can see that it leans a bit more to smaller populations than the overall population histogram.


<div>
<img src="./img/femalepop-hist.png" width="600">
</div>

Finally, I created a histogram of Average Income, as this could be a key indicator for my analysis. We can easily see that the vast majority of neighbourhoods have income under \\$112,630, and the majority under \\$73,728. There is 1 outlier with a very high average income, with over \\$400,000/year.

<div>
<img src="./img/income-hist.png" width="600">
</div>

I created a table listing the top 10 venues per neighbourhood:

<div>
<img src="./img/top-10-venues.png" width="1000">
</div>

### 3.2 Cluster Modeling

As I had unlabeled data that I wanted to be able to group by a set of characteristics (demographics and venue information), using the K Means clustering algorithm was a good choice for this problem.

The first step in K Means is determining the proper K value to use, as this can have a big impact on the output of the model. I used the Elbow Method to determine the best value for K. 

When looking at my dataset using all possible venue categories, the output of this test was K=5, as seen below.

I then ran the K Means model on my all-venues dataset.

<div>
<img src="./img/elbow-all-venues.png" width="700">
</div>

I also needed to find the optimal value for K for my salons-only dataset, as it could be different. I used the elbow method once again and found a value of K = 5 once again as seen below. I was then able to run the K Means model on my salons-only dataset.

<div>
<img src="./img/elbow-salons.png" width="700">
</div>

## 4. Results

### 4.1 All Venues Dataset

Using the value of K = 5, I was able to run the K Means model on my all-venues dataset to return cluster labels for each of my neighbourhoods. I then inserted those labels into my results:

<div>
<img src="./img/all-venues-result.png" width="1000">
</div>

Using the coordinates within the data, I was able to use Folium to plot the neighbourhoods on a map, using colour to differentiate between my clusters. I also placed a label on each point, so that clicking on it showed both the neighbourhood name and the cluster that was assigned by the model.

<div>
<img src="./img/map-all-venues.png" width="1000">
</div>

I also took the mean values for each of the clusters and put it in a separate table.

<div>
<img src="./img/averages-all-venues2.png" width="600">
</div>

From this, we can see that the two clusters with the highest average income and lowest populations have no nail salons or salon/barbershops. This could be viewed as an opportunity to venture into an untapped market, but it should also be considered that there might be a reason why there are no similar businesses in any of these neighbourhoods.

Cluster 2 has the highest rate of similar businesses, so neighbourhoods in that cluster can probably be dismissed, as the market might already be saturated.

That leaves us with clusters 0 and 4, the two largest clusters. Though cluster 4 has a lower average income than cluster 0, it also has a higher average female population, and less competition, with a total lack of salon/barbershops, but enough nail salons to indicate that it is possible to run a successful business in those neighbourhoods.

### 4.2 Salons-only dataset

Using the value of K = 5, I was able to run the K Means model on my salons-only dataset to return cluster labels for each of my neighbourhoods. I then inserted those labels into my results:

<div>
<img src="./img/results-salons.png" width="1000">
</div>

Using the coordinates within the data, I was able to use Folium to plot the neighbourhoods on a map, using colour to differentiate between my clusters. I also placed a label on each point, so that clicking on it showed both the neighbourhood name and the cluster that was assigned by the model.

<div>
<img src="./img/map-salons.png" width="1000">
</div>

I also took the mean values for each of the clusters and put it in a separate table.

<div>
<img src="./img/averages-salons.png" width="600">
</div>

## 5 Discussion

### 5.1 All Venues Dataset

In the map in section 4.1, we can see that downtown Toronto and the outskirts are mostly made up of two clusters (red(0) and orange(4) on the map), with the other three clusters (blue(2), purple(1), and light green(3)) forming a kind of ring in the suburbs surrounding downtown. This implies that the suburbs are quite different from both the downtown and the outskirts, possibly because of the average income of the residents who live there, as suburbs tend to have full homes rather than just apartments.

The averages table in section 4.1, we can see that the two clusters with the highest average income and lowest populations have no nail salons or salon/barbershops. This could be viewed as an opportunity to venture into an untapped market, but it should also be considered that there might be a reason why there are no similar businesses in any of these neighbourhoods.

Cluster 2 has the highest rate of similar businesses, so neighbourhoods in that cluster can probably be dismissed, as the market might already be saturated.

That leaves us with clusters 0 and 4, the two largest clusters. Though cluster 4 has a lower average income than cluster 0, it also has a higher average female population, and less competition, with a total lack of salon/barbershops, but enough nail salons to indicate that it is possible to run a successful business in those neighbourhoods.

### 5.2 Salons-Only Datatset

Very similarly to the all-venues dataset, in section 4.2, we can see on the map that downtown Toronto and the outskirts are mostly made up of two clusters (red(0) and orange(4) on the map), with the other three clusters (blue(2), purple(1), and light green(3)) forming a kind of ring in the suburbs surrounding downtown.

In the averages table of section 4,2, it is once again very similar to the all-venues dataset, with the only difference being clusters 1 and 2, which we had discarded for various reasons in the previous section as being ill-suited for my new business.

This largely confirmes that clusters 0 and 4 are the most suitable for a new nail salon, with cluster 4 being the most promising due to the lack of steep competition.

## 6 Conclusions

From the clustering analysis done in this report, it is likely that a neighbourhood within Cluster 4 would be the most suitable location for a new nail salon business. This is because there are other nail salons in the cluster, implying that the business type is viable, but doesn't have the highest occurence of either nail salons or salons/barbershops, meaning that there will be less stiff competition for the new business.

A next step would be to take the neighbourhoods of Cluster 4 and find more data on the area, maybe economic data about businesses, and perform another cluster analysis to break down the cluster even further.